In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

sport = pd.read_excel('data_files/Days.xlsx',sheet_name='health')

In [38]:
sport = sport.loc[:,['DATE', 'SPORT_TIME']]

In [39]:
sport = sport.dropna()
sport = sport.tail(50)
sport = sport.reset_index()

In [40]:
sport

,index,DATE,SPORT_TIME
0,5034,2022-10-14,129.0
1,5035,2022-10-15,70.0
2,5036,2022-10-16,21.0
3,5037,2022-10-17,38.0
4,5038,2022-10-18,70.0
...,...,...,...
145,5179,2023-03-08,16.0
146,5180,2023-03-09,15.0
147,5181,2023-03-10,50.0
148,5182,2023-03-11,16.0


In [41]:
# Make sure the 'DATE' column is in datetime format
sport['DATE'] = pd.to_datetime(sport['DATE'])

# Set the 'DATE' column as the index
sport = sport.set_index('DATE')

# Resample the dataframe to daily frequency, filling missing dates with zeros
df_daily = sport.resample('D').sum().fillna(0)

In [42]:
# Create a function to normalize the Sport values and map them to a colormap
def map_activity_to_color(value, cmap, vmin, vmax):
    normalized_value = (value - vmin) / (vmax - vmin)
    return cmap(normalized_value)

In [43]:
# Create the diagram with small squares

def create_activity_diagram(df, figsize=(10, 10), cmap=plt.cm.Blues, vmax=None, output_file=None):
    # Find the date range and create an empty array
    days = (df.index.max() - df.index.min()).days + 1
    weeks = int(np.ceil(days / 7))
    data = np.zeros((weeks, 7))

    # Fill the data array with Sport values
    start_date = df.index.min()
    for i in range(weeks):
        for j in range(7):
            current_date = start_date + timedelta(weeks=i, days=j)
            if current_date in df.index:
                data[i, j] = df.loc[current_date, 'SPORT_TIME']

    if vmax is None:
        vmax = data.max()

    # Create the diagram
    fig, ax = plt.subplots(figsize=figsize)
    for i in range(weeks):
        for j in range(7):
            value = data[i, j]
            color = map_activity_to_color(value, cmap, 0, vmax)
            rect = plt.Rectangle((j, i), 1, 1, facecolor=color, edgecolor='black')
            ax.add_patch(rect)

    ax.set_xlim(0, 7)
    ax.set_ylim(0, weeks)
    ax.set_yticks(np.arange(0, weeks) + 0.5)
    ax.set_yticklabels([start_date + timedelta(weeks=i) for i in range(weeks)], rotation=90, ha='right')
    ax.set_xticks(np.arange(0, 7) + 0.5)
    ax.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
    ax.invert_yaxis()

    if output_file:
        plt.savefig(output_file, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()


# Create the diagram
create_activity_diagram(df_daily, output_file='activity_diagram.jpg')
